<a href="https://colab.research.google.com/github/kittimaxz/Project_BoneAge/blob/main/Model_Bonet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:
!git clone https://github.com/BCV-Uniandes/Bonet.git
!cd Bonet

fatal: destination path 'Bonet' already exists and is not an empty directory.


In [50]:
# Load in relevant libraries, and alias where appropriate
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [51]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv) # เป็น library ในการจัดการ dataframe

In [52]:
from google.colab import drive # เชื่อม drive ของเรา ถ้าเชื่อมสำเร็จจะขึ้นคำว่าMounted at /content/drive 
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [53]:
import os # os จัดการเกี่ยวกับไฟล์ต่างๆ ในโฟล์เดอร์

In [54]:
path = '/content/drive/My Drive/Project_Boneage'  # ชี้ว่า data เราอยู่โฟล์เดอร์ไหน

In [55]:
#loading dataframes # เอาที่อยู่ของไฟล์เชื่อมกับที่อยู่ของไฟล์เรา โดยกำหนดชื่่อไฟล์ที่เราต้องการ
train_path = os.path.join(path,'train.csv')
train_dataset = pd.read_csv(train_path)
validation_path = os.path.join(path,'validation.csv')
validation_dataset = pd.read_csv(validation_path)
test_path = os.path.join(path,'boneage-test-dataset.csv')
test_dataset = pd.read_csv(test_path)

In [56]:
train_dataset

,id,boneage,male
0,1377,180,False
1,1378,12,False
2,1379,94,False
3,1380,120,True
4,1381,82,False
...,...,...,...
10084,13259,94,False
10085,13261,94,False
10086,13274,94,False
10087,13286,94,False


In [57]:
validation_dataset

,id,boneage,male
0,10186,10,True
1,10758,15,False
2,10860,180,False
3,10877,180,False
4,11008,180,False
...,...,...,...
2517,15605,50,False
2518,15606,113,False
2519,15608,55,False
2520,15609,150,True


In [58]:
test_dataset

,Case ID,Sex,Ground truth bone age (months)
0,4360,M,169
1,4361,M,170
2,4362,M,73
3,4363,M,153
4,4364,M,135
...,...,...,...
195,4555,F,133
196,4556,F,129
197,4557,F,168
198,4558,F,136


In [59]:
#appending file extension to id column for both training and testing dataframes
# ต่อท้าย data ใน column 'id' ด้วย .png
train_dataset['id'] = train_dataset['id'].apply(lambda x: str(x)+'.png') # ทำการเปลี่ยน type ใน column 'id' จาก int  เป็น string เพื่อให้สามารถต่อท้ายด้วย .png ได้ # ฟังก์ชันแลมบ์ดาเป็นฟังก์ชันที่ไม่ระบุตัวตนขนาดเล็ก
validation_dataset['id'] = validation_dataset['id'].apply(lambda x: str(x)+'.png')
test_dataset['Case ID'] = test_dataset['Case ID'].apply(lambda x: str(x)+'.png')

train_dataset.head() # แสดงค่าของ train_df บางส่วน

,id,boneage,male
0,1377.png,180,False
1,1378.png,12,False
2,1379.png,94,False
3,1380.png,120,True
4,1381.png,82,False


In [60]:
#finding out the number of male and female children in the dataset
#creating a new column called gender to keep the gender of the child as a string
train_dataset['gender'] = train_dataset['male'].apply(lambda x: 'male' if x else 'female') # ทำการเปลี่ยนค่าข้อมูล gender จาก true -> male, false -> female 
train_dataset['gender']

0        female
1        female
2        female
3          male
4        female
          ...  
10084    female
10085    female
10086    female
10087    female
10088    female
Name: gender, Length: 10089, dtype: object

In [61]:
validation_dataset['gender'] = validation_dataset['male'].apply(lambda x: 'male' if x else 'female') # ทำการเปลี่ยนค่าข้อมูล gender จาก true -> male, false -> female 
validation_dataset['gender']

0         male
1       female
2       female
3       female
4       female
         ...  
2517    female
2518    female
2519    female
2520      male
2521      male
Name: gender, Length: 2522, dtype: object

In [62]:
#mean age is # หาค่าเฉลี่ยอายุกระดูก
mean_bone_age = train_dataset['boneage'].mean() 
print('mean: ' + str(mean_bone_age))

mean: 127.29467737139458


In [63]:
#standard deviation of boneage # หาค่าส่วนเบี่ยงเบนมาตรฐานของอายุกระดูก
std_bone_age = train_dataset['boneage'].std()
std_bone_age

41.230590949837755

In [64]:
#models perform better when features are normalised to have zero mean and unity standard deviation
#using z score for the training 
# ทำการหาค่า z-score 
train_dataset['bone_age_z'] = (train_dataset['boneage'] - mean_bone_age)/(std_bone_age)
train_dataset

,id,boneage,male,gender,bone_age_z
0,1377.png,180,False,female,1.278306
1,1378.png,12,False,female,-2.796338
2,1379.png,94,False,female,-0.807524
3,1380.png,120,True,male,-0.176924
4,1381.png,82,False,female,-1.098570
...,...,...,...,...,...
10084,13259.png,94,False,female,-0.807524
10085,13261.png,94,False,female,-0.807524
10086,13274.png,94,False,female,-0.807524
10087,13286.png,94,False,female,-0.807524


In [65]:
import matplotlib.pyplot as plt # เป็น module พื้นฐานที่ใช้ในการสร้างกราฟของ python
import matplotlib.image as mpimg #นำเข้า library ที่ใช้อ่านรูป

In [66]:
trian_imgs = os.path.join(path,'boneage-training-dataset') #เอาพาร์ทที่อยู่ในไฟล์มาเชื่อมกับColab
print(trian_imgs)

/content/drive/My Drive/Project_Boneage/boneage-training-dataset


In [67]:
validation_imgs = os.path.join(path,'boneage_validation_dataset') #เอาพาร์ทที่อยู่ในไฟล์มาเชื่อมกับColab
print(validation_imgs)

/content/drive/My Drive/Project_Boneage/boneage_validation_dataset


In [68]:
mean_boneage = validation_dataset['boneage'].mean() 
print('mean: ' + str(mean_boneage))

mean: 127.42505947660587


In [69]:
std_boneage = validation_dataset['boneage'].std()
std_boneage

40.99513006091403

In [70]:
validation_dataset['bone_age_z'] = (validation_dataset['boneage'] - mean_boneage)/(std_boneage)
validation_dataset

,id,boneage,male,gender,bone_age_z
0,10186.png,10,True,male,-2.864366
1,10758.png,15,False,female,-2.742400
2,10860.png,180,False,female,1.282468
3,10877.png,180,False,female,1.282468
4,11008.png,180,False,female,1.282468
...,...,...,...,...,...
2517,15605.png,50,False,female,-1.888640
2518,15606.png,113,False,female,-0.351873
2519,15608.png,55,False,female,-1.766675
2520,15609.png,150,True,male,0.550674


In [71]:
#library required for image preprocessing 
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from  keras.applications.inception_v3 import preprocess_input 

In [72]:
#reducing down the size of the image 
img_size = 500 # ลดขนาดรูปภาพ

In [73]:
train_data_generator = ImageDataGenerator(preprocessing_function = preprocess_input)
val_data_generator = ImageDataGenerator(preprocessing_function = preprocess_input)

In [74]:
#train data generator
train_generator = train_data_generator.flow_from_dataframe(
    dataframe = train_dataset,
    directory = (os.path.join(path,'boneage_training_dataset/')),
    x_col= 'id',
    y_col= 'bone_age_z',
    class_mode= 'raw',
    start_epoch=0,
    num_epochs=60,
    lr=0.0001,
    patience=2,
    batch_size=20,
    num_workers=0,
    num_gpus=1,
    gpus=0,
    target_size = (img_size, img_size))

Found 10089 validated image filenames.


In [75]:
#validation data generator
val_generator = val_data_generator.flow_from_dataframe(
    dataframe = validation_dataset,
    directory = (os.path.join(path,'boneage_validation_dataset/')),
    x_col = 'id',
    y_col = 'bone_age_z',
    class_mode = 'raw',
    start_epoch=0,
    num_epochs=60,
    lr=0.0001,
    patience=2,
    batch_size=20,
    num_workers=0,
    num_gpus=1,
    gpus=0,
    target_size = (img_size, img_size))

Found 2522 validated image filenames.


In [76]:
#test data generator
test_data_generator = ImageDataGenerator(preprocessing_function = preprocess_input)
test_generator = test_data_generator.flow_from_directory(
    directory = (os.path.join(path,'boneage-test-dataset/')),
    target_size = (img_size,img_size),
    class_mode = 'binary',classes=['.'])

Found 200 images belonging to 1 classes.


In [77]:
import re
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.utils.checkpoint as cp
from collections import OrderedDict
import torch.utils.model_zoo as model_zoo
from torch.utils.model_zoo import load_url as load_state_dict_from_url

In [78]:
class BoNet(nn.Module):

    def __init__(self, num_class = 1):
        super(BoNet, self).__init__()
        
        # Backbone
        self.inception_v3 = InceptionV3()

        # Gender
        self.gender = DenseLayer(1, 32)
        self.fc_1 = DenseLayer(100384, 1000)
        self.fc_2 = DenseLayer(1000, 1000)
        self.fc_3 = nn.Linear(1000, 1)

    def forward(self, x_col, y_col):
        x = self.inception_v3(x_col)
        y = self.gender(y_col)
        x = self.fc_1(torch.cat([x_col, y_col], 1))
        x = self.fc_2(x_col)
        x = self.fc_3(x_col)
        return x

In [79]:
class DenseLayer(nn.Module):

    def __init__(self, in_channels, out_channels):
        super(DenseLayer, self).__init__()
        self.linear = nn.Linear(in_channels, out_channels)

    def forward(self, x):
        x = self.linear(x)
        return F.relu(x, inplace=True)

In [80]:
import Bonet
from Bonet.models.bonet import BoNet

In [81]:
num_classes = 1

In [82]:
model = BoNet(num_classes) 

In [83]:
net = BoNet()

In [84]:
# Set Loss function with criterion
criterion = nn.L1Loss()

In [85]:
# Set optimizer with optimizer
optimizer = optim.Adam(net.parameters(), lr = 0.0001 * 500 )
annealing = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, factor=0.8, patience= 2, cooldown=5,
    min_lr=0.00001, eps=0.00001, verbose=True)

In [86]:
import time
!pip install horovod
import horovod.torch as hvd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [87]:
# Device will determine whether to run the training on GPU or CPU.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [88]:
start_epoch = 0
epoch = 60
distributed = []
rank = 0

In [89]:
def main():
    print('Train begins...')
    best_val_loss = None
    # Find best model in validation
    if osp.exists(osp.join(save_folder, 'train.csv')):
        with open(osp.join(save_folder, 'train.csv')) as csv_file:
            csv_reader = csv.reader(csv_file, delimiter=',')
            val_list = []
            for row in csv_reader:
                val_list.append(float(row[2]))
            best_val_loss = min(val_list)
    if eval_first:
        val_loss = evaluate()
    try:
        out_file = open(os.path.join(save_folder, 'train.csv'), 'a+')
        
        for epoch in range(start_epoch, epoch + 1):
            if distributed:
                train_dataset.set_epoch(epoch)
                validation_dataset.set_epoch(epoch)
            if rank == 0:
                epoch_start_time = time.time()
            train_loss = train_dataset(epoch)
            annealing.step(train_loss)
            val_loss = evaluate()
            if rank == 0:
                print('-' * 89)
                print('| end of epoch {:3d} | time: {:5.2f}s '
                      '| epoch loss {:.6f} |'.format(
                          epoch, time.time() - epoch_start_time, train_loss))
                print('-' * 89)
                out_file.write('{}, {}, {}\n'.format(epoch, train_loss, val_loss))
                out_file.flush()

                if best_val_loss is None or val_loss > best_val_loss and rank == 0:
                    best_val_loss = val_loss
                    filename = osp.join(save_folder, 'boneage_bonet_weights.pth')
                    torch.save(net.state_dict(), filename)
        out_file.close()
    except KeyboardInterrupt:
        print('-' * 89)
        print('Exiting from training early')

In [90]:
def train(epoch):
    net.train()
    total_loss = AverageMeter()
    epoch_loss_stats = AverageMeter()
    time_stats = AverageMeter()
    loss = 0
    optimizer.zero_grad()
    for (batch_idx, (train_imgs, bone_age, gender, _)) in enumerate(train_dataset):
        imgs = train_imgs.to(device)
        bone_ages = bone_age.to(device)
        genders = gender.to(device)

        start_time = time.time()
        outputs = net(train_imgs, gender)
        loss = criterion(outputs.squeeze(), bone_age)
        loss.backward()
        optimizer.step()

        loss = metric_average(loss.item(), 'loss')

        time_stats.update(time.time() - start_time, 1)
        total_loss.update(loss, 1)
        epoch_loss_stats.update(loss, 1)
        optimizer.zero_grad()

        if (batch_idx % log_interval == 0) and rank == 0:
            elapsed_time = time_stats.avg
            print(' [{:5d}] ({:5d}/{:5d}) | ms/batch {:.4f} |'
                  ' loss {:.6f} | avg loss {:.6f} | lr {:.7f}'.format(
                      epoch, batch_idx, len(train_dataset),
                      elapsed_time * 1000, total_loss.avg,
                      epoch_loss_stats.avg,
                      optimizer.param_groups[0]['lr']))
            total_loss.reset()

    epoch_total_loss = epoch_loss_stats.avg
    args.resume_iter = 0

    if args.rank == 0:
        filename = 'boneage_bonet_snapshot.pth'
        filename = osp.join(args.save_folder, filename)
        torch.save(net.state_dict(), filename)

        optim_filename = 'boneage_bonet_optim.pth'
        optim_filename = osp.join(args.save_folder, optim_filename)
        torch.save(optimizer.state_dict(), optim_filename)

    return epoch_total_loss

In [91]:
def evaluate():
    net.eval()
    epoch_total_loss = AverageMeter()
    for (batch_idx, (validation_imgs, bone_age, gender, _)) in enumerate(validation_dataset):
        imgs = validation_imgs.to(device)
        bone_ages = bone_age.to(device)
        genders = gender.to(device)

        with torch.no_grad():
            outputs = net(validation_imgs, gender)
        loss = criterion(outputs.squeeze(), bone_age)
        loss = metric_average(loss.item(), 'loss')
        epoch_total_loss.update(loss, 1)

    epoch_total_loss = epoch_total_loss.avg

    if rank == 0:
        print('Val loss: {:.5f}'.format(epoch_total_loss))

    return epoch_total_loss


In [92]:
def metric_average(val, name):
    tensor = torch.tensor(val)
    avg_tensor = hvd.allreduce(tensor, name=name)
    return avg_tensor.item()

In [93]:
class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [94]:
net.state_dict()

OrderedDict([('inception_v3.Conv2d_1a_3x3.conv.weight',
              tensor([[[[-0.0959,  0.0756, -0.0381],
                        [ 0.0010,  0.0633, -0.1204],
                        [-0.0185, -0.1058,  0.0537]]],
              
              
                      [[[ 0.1430, -0.0093, -0.1156],
                        [-0.1834, -0.0756, -0.0210],
                        [ 0.1042, -0.0804,  0.1476]]],
              
              
                      [[[-0.1958, -0.0161,  0.0656],
                        [ 0.0921,  0.0413,  0.0162],
                        [ 0.1533,  0.0857,  0.1529]]],
              
              
                      [[[ 0.0028,  0.0841, -0.1131],
                        [-0.0052,  0.0205,  0.1110],
                        [ 0.0487, -0.0195, -0.0375]]],
              
              
                      [[[ 0.0200,  0.0800, -0.0556],
                        [ 0.0729,  0.0172,  0.1126],
                        [-0.0978,  0.1651,  0.1953]]],
              
    